In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "../Project Schedule/cee240_sched_dur.xlsx"
# import csv 
df = pd.read_excel(path, names=["activities", "durations"], usecols=[3,4])

In [3]:
df

,activities,durations
0,NTP,0.0
1,submit shut down request for facilities,1.0
2,"demo ceilings GL 6.5, M.4",2.0
3,"erect temp barrier, GL 6.5, M.4",3.0
4,demo exterior walls GL 5.7 M,3.0
...,...,...
187,protect trees Courtyard,2.0
188,install light fixtures Courtyard,5.0
189,install resin paving Courtyard,6.0
190,install courtyard furnishings Courtyard,5.0


# Make P6 Dataframe

## Tasks

In [4]:
p6_path = "../../Documents/cee240_p6/CEE240F22_AandR_All.xlsx"
# import csv 
df_task = pd.read_excel(p6_path, usecols=[0,1,2,3,4,5])

In [5]:
df_task

,task_code,status_code,wbs_id,target_drtn_hr_cnt,task_name,delete_record_flag
0,Activity ID,Activity Status,WBS Code,Original Duration(h),Activity Name,Delete This Row


In [6]:
row_nums = [[3,59], [62,87], [89,113], [115,132], [134,150], [152,169], [172, 186], [188,193]]

In [7]:
row_nums_acc = [(pair[0] -3, pair[1] - 2) for pair in row_nums]
row_nums_acc   

[(0, 57),
 (59, 85),
 (86, 111),
 (112, 130),
 (131, 148),
 (149, 167),
 (169, 184),
 (185, 191)]

In [8]:
act_prefixes = ["P1B", "BGL", "B1L", "B2L", "AGL", "A1L",  "P2A1L", "C"]
wbs_names = ["CEE240F22.Inc01", "CEE240F22.Inc02.B_LG", "CEE240F22.Inc02.B_L1", "CEE240F22.Inc02.B_L2", "CEE240F22.Inc02.A_LG", "CEE240F22.Inc02.A_L1", "CEE240F22.Inc02.Ph2_A_L1", "CEE240F22.Inc02.Courtyard" ]

In [9]:
len(act_prefixes)

8

In [10]:
len(wbs_names)

8

In [11]:
def add_tasks(df_task, row, act_prefix, wbs):
    inc = df["activities"].iloc[row[0]:row[1]]
    inc_act_id = [f"{act_prefix}{1000 + i*10}" for i in range(len(inc))]
    inc_dict = {
    "task_code":inc_act_id,
    "wbs_id": wbs,
    "target_drtn_hr_cnt": df["durations"].iloc[row[0]:row[1]],
    "task_name": inc
    }
    df_task = pd.concat([df_task, pd.DataFrame(inc_dict)], ignore_index=True)
    # print(df_task)
    return df_task

In [12]:
for row, act_prefix, wbs in zip(row_nums_acc, act_prefixes, wbs_names):
    df_task = add_tasks(df_task, row, act_prefix, wbs)

In [13]:
df_task

,task_code,status_code,wbs_id,target_drtn_hr_cnt,task_name,delete_record_flag
0,Activity ID,Activity Status,WBS Code,Original Duration(h),Activity Name,Delete This Row
1,P1B1000,NaN,CEE240F22.Inc01,0.0,NTP,NaN
2,P1B1010,NaN,CEE240F22.Inc01,1.0,submit shut down request for facilities,NaN
3,P1B1020,NaN,CEE240F22.Inc01,2.0,"demo ceilings GL 6.5, M.4",NaN
4,P1B1030,NaN,CEE240F22.Inc01,3.0,"erect temp barrier, GL 6.5, M.4",NaN
...,...,...,...,...,...,...
178,C1010,NaN,CEE240F22.Inc02.Courtyard,6.0,install concrete paving Courtyard,NaN
179,C1020,NaN,CEE240F22.Inc02.Courtyard,2.0,protect trees Courtyard,NaN
180,C1030,NaN,CEE240F22.Inc02.Courtyard,5.0,install light fixtures Courtyard,NaN
181,C1040,NaN,CEE240F22.Inc02.Courtyard,6.0,install resin paving Courtyard,NaN


## Relationships

In [14]:
df_pred = pd.read_excel(p6_path, sheet_name="TASKPRED")

In [15]:
df_pred

,pred_task_id,task_id,pred_type,PREDTASK__status_code,TASK__status_code,pred_proj_id,proj_id,predtask__projwbs__wbs_full_name,task__projwbs__wbs_full_name,predtask__task_name,task__task_name,delete_record_flag
0,Predecessor,Successor,Relationship Type,(*)Predecessor Activity Status,(*)Successor Activity Status,(*)Predecessor Project,(*)Successor Project,(*)Predecessor WBS,(*)Successor WBS,(*)Predecessor Activity Name,(*)Successor Activity Name,Delete This Row


In [16]:
dict_pred = {
    "pred_task_id" : df_task["task_code"][2:],
    "task_id": df_task["task_code"][1:],
    "pred_type": "FS"

}

In [17]:
df_pred = pd.concat([df_pred, pd.DataFrame(dict_pred)], ignore_index=True)

In [18]:
df_pred

,pred_task_id,task_id,pred_type,PREDTASK__status_code,TASK__status_code,pred_proj_id,proj_id,predtask__projwbs__wbs_full_name,task__projwbs__wbs_full_name,predtask__task_name,task__task_name,delete_record_flag
0,Predecessor,Successor,Relationship Type,(*)Predecessor Activity Status,(*)Successor Activity Status,(*)Predecessor Project,(*)Successor Project,(*)Predecessor WBS,(*)Successor WBS,(*)Predecessor Activity Name,(*)Successor Activity Name,Delete This Row
1,NaN,P1B1000,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P1B1010,P1B1010,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P1B1020,P1B1020,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P1B1030,P1B1030,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
178,C1010,C1010,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,C1020,C1020,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,C1030,C1030,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,C1040,C1040,FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# with pd.ExcelWriter("../../Documents/cee240_p6/CEE240F22_toP62.xlsx") as writer: 
#     df_task.to_excel(writer, sheet_name="TASK", index=False)
#     df_pred.to_excel(writer, sheet_name="TASKPRED", index=False)

In [20]:
with pd.ExcelWriter("outputs/CEE240F22_toP62.xlsx") as writer: 
    df_task.to_excel(writer, sheet_name="TASK", index=False)
    df_pred.to_excel(writer, sheet_name="TASKPRED", index=False)